In [1]:
import torch
import numpy as np
from mlagents_envs.environment import UnityEnvironment
from mlagents_envs.base_env import ActionTuple


In [2]:
if(torch.cuda.is_available()):
    device = torch.device("cuda")
    print(device, torch.cuda.get_device_name(0))
else:
    device= torch.device("cpu")
    print(device)

cpu


In [4]:
fname = "3DBall-119867.pth"   
#檔名同最後一次train的pth檔
MyModel = torch.load(fname)
MyModel.eval()
MyModel.to(device)

SimpleActor(
  (network_body): NetworkBody(
    (observation_encoder): ObservationEncoder(
      (processors): ModuleList(
        (0): VectorInput(
          (normalizer): Normalizer()
        )
      )
    )
    (processors): ModuleList(
      (0): VectorInput(
        (normalizer): Normalizer()
      )
    )
    (_body_endoder): LinearEncoder(
      (seq_layers): Sequential(
        (0): Linear(in_features=8, out_features=128, bias=True)
        (1): Swish()
        (2): Linear(in_features=128, out_features=128, bias=True)
        (3): Swish()
      )
    )
  )
  (action_model): ActionModel(
    (_continuous_distribution): GaussianDistribution(
      (mu): Linear(in_features=128, out_features=2, bias=True)
    )
  )
)

In [5]:
MyEncoder = MyModel.network_body._body_endoder

In [6]:
MyActionModel = MyModel.action_model

In [7]:
distribution = MyActionModel._continuous_distribution
print(distribution)

GaussianDistribution(
  (mu): Linear(in_features=128, out_features=2, bias=True)
)


# Connect to Unity

In [57]:
# Run this line and go to Unity editor to click Play
#這一行往下需要到unity按下play
env = UnityEnvironment(file_name= None, base_port=5004) 
env.reset()

In [58]:
behaviorNames = list(env.behavior_specs.keys())
behaviorName = behaviorNames[0]
print(behaviorName)

3DBall?team=0


In [59]:
DecisionSteps, TerminalSteps = env.get_steps(behaviorName)
s = DecisionSteps.obs[0]
print(s.shape)
#(??? ,可動作種類)

(1, 8)


In [60]:
s = torch.FloatTensor(s).to(device)

In [61]:
s1 = MyEncoder(s)
print(s1.shape)

torch.Size([1, 128])


In [62]:
print( distribution(s1).mean, distribution(s1).std)

tensor([[-0.4161,  0.7225]], grad_fn=<AddmmBackward0>) tensor([[0.9107, 0.8885]], grad_fn=<ExpBackward0>)


In [63]:
a = distribution(s1).sample()
print(a)

tensor([[-0.5948, -0.3757]], grad_fn=<AddBackward0>)


In [64]:
a = a.cpu().detach().numpy()
print(a)

[[-0.59482473 -0.3756724 ]]


In [65]:
a = ActionTuple(np.array(a, dtype=np.float32))
print(a)

In [66]:
env.set_actions(behaviorName, a)
env.step()

In [55]:
DecisionSteps, TerminalSteps = env.get_steps(behaviorName)
DecisionSteps.reward

array([0.1], dtype=float32)

In [67]:
env.close()

### Try to change prob.

In [76]:
from torch.distributions import Normal
dist1  = Normal(5, 0.5)
print(dist1)

Normal(loc: 5.0, scale: 0.5)


In [42]:
a = dist1.sample()
print(a)

tensor(4.6559)


In [43]:
mean1 = distribution(s1).mean
mean1 = mean1.cpu().detach().numpy()
print(mean1, mean1[0], mean1[0][0])

[[-0.49471715  0.74056077]] [-0.49471715  0.74056077] -0.49471715


In [44]:
std1 = distribution(s1).std
std1 = std1.cpu().detach().numpy()
print(std1, std1[0], std1[0][0])

[[0.9107054 0.8885418]] [0.9107054 0.8885418] 0.9107054


In [45]:
m11, m12 = mean1[0][0], mean1[0][1]
std11, std12 = std1[0][0], std1[0][1]
a11 = float(Normal(m11, std11).sample())
a12 = float(Normal(m12, std12).sample())
a1 = [[a11, a12]]
print(a1)

[[-1.5019490718841553, 2.5863988399505615]]


In [46]:
a = ActionTuple(np.array(a1, dtype=np.float32))
print(a)

In [47]:
env.set_actions(behaviorName, a)
env.step()